# Assignment 1000:Fine Tuning an LLM for Text Generation

### Instrctor: Dr. Ankur Mali

### Submission by: Abrar Zahin


### Objective:
The objective of this notebook is to fine tune an open source pre-trained LLM on a dataset to learn about the significance of this process. After fine tunign the model,

In [1]:
!pip install --quiet transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12

In [2]:

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import load_dataset

model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

dataset = load_dataset("xsum")
trust_remote_code = True

train_data = dataset["train"]       #train_data derived by splitting the dataset
val_data = dataset["validation"]      #Val_data is validation data derived by splitting the dataset


def tokenize_function(example):

  #tokenizer.add_special_tokens({'pad_token': '[PAD]'})
  tokenizer.pad_token = tokenizer.eos_token    #adding a padding token to the tokenizer, this is a default padding token that the tokenizer uses.
  result = tokenizer(
    example["summary"],
    truncation=True,
    padding="max_length", # could also use ’longest’
    max_length=128  # set a reasonable max length
  )
  result["labels"] = result["input_ids"].copy()
  return result

model.resize_token_embeddings(len(tokenizer))
tokenized_train = train_data.map(tokenize_function, batched=True)   #map the tokenizer function to each example in the dataset
tokenized_val = val_data.map(tokenize_function, batched=True)
# Remove columns other than input_ids/attention_mask
tokenized_train.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_val.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

Map:   0%|          | 0/204045 [00:00<?, ? examples/s]

Map:   0%|          | 0/11332 [00:00<?, ? examples/s]

In [ ]:
prompt = "An Australian man has been rescued after surviving six days in the outback by eating ants. The man became lost after his car broke down in a remote part of Western Australia. Authorities say he was found by a farm worker, suffering from exposure, but otherwise in good condition."
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda()

output = model.generate(input_ids, max_length=100, num_beams=5, no_repeat_ngram_size=2, early_stopping=True)
response = tokenizer.decode(output[0], skip_special_tokens=True)
print(f'Prompt: {prompt}\nResponse: {response}')

In [ ]:
training_args = TrainingArguments(
    output_dir="./finetuned_gpt2xsum",
    eval_strategy="epoch",
    num_train_epochs=1, # Increase for better results
    per_device_train_batch_size=1, #Adjust based on GPU memory
    per_device_eval_batch_size=1,
    gradient_accumulation_steps = 4,
    save_steps=500,
    logging_strategy="steps",
    logging_steps=100,
    #load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: abrarzahin2511 (abrarzahin2511-university-of-south-florida) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss


In [ ]:
prompt = "An Australian man has been rescued after surviving six days in the outback by eating ants. The man became lost after his car broke down in a remote part of Western Australia. Authorities say he was found by a farm worker, suffering from exposure, but otherwise in good condition."
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda()

output = model.generate(input_ids, max_length=100, num_beams=5, no_repeat_ngram_size=2, early_stopping=True)
response = tokenizer.decode(output[0], skip_special_tokens=True)
print(f'Prompt: {prompt}\nResponse: {response}')

# Challenges

I faced x main challanges while conducting the training. One of them was **running out of GPU memory**. While training the model, to decrease the loss, I aimed to train the model on 3 epochs. The GPU almost ran out of memory.
To mitigate these issues, I reduced the batch_size and used gradient accumulation.